## PPG Signal Analysis and Feature Extraction

In this notebook, you will explore and extract meaningful features from PPG (Photoplethysmogram) signals from the temporal and spectral domain.
This task will be performed using the common libraries for these tasks, HeartPy and NeuroKit2, respectively.

To get started, install heartpy and neurokit2. To do it, you can run the following command on the vscode terminal.
- pip install opencv-python numpy scikit-image


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import os
import cv2
from skimage.measure import label, regionprops
from skimage.color import rgb2gray
from skimage.segmentation import clear_border
from skimage import filters
from sklearn.cluster import KMeans

### Features Extraction

To extract the features of images, we use computer vision techniques that are provided by a diversity of libraries.
Because there are multiple ways of extracting information, feel free to explore all the techniques you can find.
We suggest to search for ways to do it by researching a bit and discuss techniques with your favorite Chatbot, so you can understand the technique you are using, why you are using it, etc.

In the extraction below, we work as follows:
1. Conver the image into a structure of np.array (so, 3 images 2D images, one for each color value of RGB, since each pixel is determined as (Red, Green, Blue))
2. Segment the image using the Otsu technique, labeling it, creating a True/False Mask and filling any holes inside the segmented area. This way, we can work with only the skin lesion in the image!
3. Create functions that extract from the skin lecion the features. They are described in the code itself.
4. Return a dictionary with the features extracted.

In [ ]:
from scipy.ndimage import binary_fill_holes
from skimage.measure import perimeter, label, regionprops

def extract_abcd_features(pil_image):
    # Convert PIL image to OpenCV (numpy array)
    image = np.array(pil_image)
    
    # Resize for consistency
    image = cv2.resize(image, (900, 900))
    
    # Grayscale & thresholding
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Clean borders & label
    labeled = label(thresh)
    regions = regionprops(labeled)
    
    if not regions:
        print("No lesion found!")
        return
    else:
        # Get the region with the largest area
        region = max(regions, key=lambda r: r.area)
        for i, r in enumerate(regions):
            print(f"Region {i}: area = {r.area}")
        lesion_mask = labeled == region.label
        lesion_mask = binary_fill_holes(lesion_mask)
    
    ### A - Asymmetry
    def compute_aligned_asymmetry(mask, target_size=(512, 512)):
        # 1. Find bounding box of lesion
        coords = np.argwhere(mask)
        if coords.size == 0:
            print("No lesion found.")
            return 0
        y0, x0 = coords.min(axis=0)
        y1, x1 = coords.max(axis=0) + 1
        cropped = mask[y0:y1, x0:x1]
        
        # 2. Resize to fixed size for standardization
        resized = cv2.resize(cropped.astype(np.uint8), target_size, interpolation=cv2.INTER_NEAREST)
        
        # 3. Vertical asymmetry (left-right)
        h, w = resized.shape
        left = resized[:, :w // 2]
        right = resized[:, w - w // 2:]
        right_flipped = np.fliplr(right)
        
        vertical_diff = np.sum(np.abs(left - right_flipped))
        
        # 4. Horizontal asymmetry (top-bottom)
        top = resized[:h // 2, :]
        bottom = resized[h - h // 2:, :]
        bottom_flipped = np.flipud(bottom)
        
        horizontal_diff = np.sum(np.abs(top - bottom_flipped))
        
        # 5. Normalize by total lesion area
        lesion_area = np.sum(resized)
        if lesion_area == 0:
            return 0
    
        asymmetry_score = (vertical_diff + horizontal_diff) / lesion_area
        return round(asymmetry_score, 4)

    ### B - Border Irregularity
    def compute_border_irregularity(mask):
        # Label connected region (we assume mask is cleaned already)
        labeled = label(mask)
        props = regionprops(labeled)

        if not props:
            return 0.0
        region = max(props, key=lambda x: x.area)

        area = region.area
        peri = perimeter(region.image)

        if area == 0:
            return 0.0
        
        irregularity = (peri ** 2) / (4 * np.pi * area)
        return round(irregularity, 4)


    ### C - Color Variation
    def compute_color_variation(original_rgb_image, lesion_mask, n_clusters=5):
        # Resize mask to match image if needed
        if lesion_mask.shape != original_rgb_image.shape[:2]:
            lesion_mask = cv2.resize(lesion_mask.astype(np.uint8), original_rgb_image.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)

        # Extract only pixels inside the lesion
        lesion_pixels = original_rgb_image[lesion_mask > 0]

        if len(lesion_pixels) < n_clusters:
            print("Error. Segmented image too small")
            return

        # Run k-means clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels = kmeans.fit_predict(lesion_pixels)
        centers = kmeans.cluster_centers_

        # Compute intra-cluster variance (sum of squared distances)
        intra_variance = np.sum((lesion_pixels - centers[labels]) ** 2) / len(lesion_pixels)

        return n_clusters, intra_variance, centers.astype(np.uint8)

    ### D - Diameter
    def compute_diameter(mask):
        region = regionprops(mask.astype(int))[0]
        diameter = max(region.major_axis_length, region.minor_axis_length)
        return diameter

    asymmetry = compute_aligned_asymmetry(lesion_mask)
    border_irregularity = compute_border_irregularity(lesion_mask)
    color_variation = compute_color_variation(image, lesion_mask)[2]
    diameter = compute_diameter(lesion_mask)

    return {
        "asymmetry": round(asymmetry, 4),
        "border_irregularity": round(border_irregularity, 4),
        "color_variation": color_variation,
        "diameter_pixels": round(diameter, 2)
    }

    
extract_abcd_features(img)

### Feature extraction auxiliar functions' structure

The feature exctraction functions have multiple steps. To aid your understanding, this block will run with visual aids for you to see what is happening at each step of the feature extraction.
This way, you can more easily understand what we are doing and modify the technique if you deem usefull.

In [ ]:
from scipy.ndimage import binary_fill_holes
from skimage.measure import label, regionprops, perimeter
from skimage.segmentation import find_boundaries
from mpl_toolkits.mplot3d import Axes3D

def visualize_abcd_features(pil_image):
    # Convert PIL image to OpenCV (numpy array)
    image = np.array(pil_image)
    
    # Resize for consistency
    image = cv2.resize(image, (900, 900))
    
    # Grayscale & thresholding
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Clean borders & label
    labeled = label(thresh)
    regions = regionprops(labeled)
    
    if not regions:
        print("No lesion found!")
        return
    else:
        # Get the region with the largest area
        region = max(regions, key=lambda r: r.area)
        for i, r in enumerate(regions):
            print(f"Region {i}: area = {r.area}")
        lesion_mask = labeled == region.label
        lesion_mask = binary_fill_holes(lesion_mask)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(thresh)
    plt.title("Threshold")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(lesion_mask, cmap='gray')
    plt.title("Lesion Mask")
    plt.axis('off')
    plt.show()
    
    ### A - Asymmetry
    def plot_lesion_symmetry(mask, target_size=(200, 200)):
        # Step 1: Get tight crop
        coords = np.argwhere(mask)
        if coords.size == 0:
            print("No lesion found.")
            return
        
        y0, x0 = coords.min(axis=0)
        y1, x1 = coords.max(axis=0) + 1
        cropped = mask[y0:y1, x0:x1]
        
        # Step 2: Resize to standard size
        resized = cv2.resize(cropped.astype(np.uint8), target_size, interpolation=cv2.INTER_NEAREST)
        
        h, w = resized.shape
        left = resized[:, :w // 2]
        right = resized[:, w - w // 2:]
        right_flipped = np.fliplr(right)
        
        top = resized[:h // 2, :]
        bottom = resized[h - h // 2:, :]
        bottom_flipped = np.flipud(bottom)
        
        # Step 3: Compute diffs
        vertical_diff = np.abs(left - right_flipped)
        horizontal_diff = np.abs(top - bottom_flipped)
        
        # Padding diffs to full size for display
        vert_map = np.hstack([left, right_flipped])
        vert_diff_padded = np.pad(vertical_diff, ((0, 0), (0, w - vertical_diff.shape[1])), mode='constant')
        
        hori_map = np.vstack([top, bottom_flipped])
        hori_diff_padded = np.pad(horizontal_diff, ((0, h - horizontal_diff.shape[0]), (0, 0)), mode='constant')
        
        # Step 4: Plot
        fig, axs = plt.subplots(2, 3, figsize=(12, 8))
        
        axs[0, 0].imshow(resized, cmap='gray')
        axs[0, 0].set_title("Aligned Lesion")
        
        axs[0, 1].imshow(vert_map, cmap='gray')
        axs[0, 1].set_title("Left vs. Flipped Right")
        
        axs[0, 2].imshow(vert_diff_padded, cmap='hot')
        axs[0, 2].set_title("Vertical Diff Map")
        
        axs[1, 1].imshow(hori_map, cmap='gray')
        axs[1, 1].set_title("Top vs. Flipped Bottom")
        
        axs[1, 2].imshow(hori_diff_padded, cmap='hot')
        axs[1, 2].set_title("Horizontal Diff Map")
        
        for ax in axs.ravel():
            ax.axis('off')
        
        plt.tight_layout()
        plt.show()

    ### B - Border Irregularity
    def plot_border_irregularity(mask, original_image=None):
        # Label regions
        labeled = label(mask)
        props = regionprops(labeled)
        
        if not props:
            print("No lesion found.")
            return

        # Largest region
        region = max(props, key=lambda x: x.area)
        region_mask = labeled == region.label

        # Perimeter & Irregularity
        area = region.area
        peri = perimeter(region.image)
        irregularity = (peri ** 2) / (4 * np.pi * area) if area > 0 else 0

        # Create full-size boundary mask
        full_boundary = np.zeros_like(mask, dtype=bool)
        
        # Get bounding box
        minr, minc, maxr, maxc = region.bbox
        cropped_mask = region.image  # cropped binary region
        cropped_boundary = find_boundaries(cropped_mask, mode='outer')

        # Place boundary back into full image
        full_boundary[minr:maxr, minc:maxc] = cropped_boundary

        # Plotting
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        axs[0].imshow(region_mask, cmap='gray')
        axs[0].set_title("Lesion Mask")
        axs[0].axis('off')

        axs[1].imshow(full_boundary, cmap='hot')
        axs[1].set_title("Detected Border")
        axs[1].axis('off')

        # Overlay boundary on original
        if original_image is not None:
            overlay = original_image.copy()
            if overlay.shape[2] == 4:
                overlay = overlay[:, :, :3]
            overlay[full_boundary] = [0, 255, 255]
            axs[2].imshow(overlay)
            axs[2].set_title("Overlay on Original")
        else:
            axs[2].imshow(region_mask, cmap='gray')
            axs[2].imshow(full_boundary, cmap='Blues', alpha=0.5)
            axs[2].set_title("Border Overlay")

        axs[2].axis('off')
        plt.suptitle(f"Border Irregularity Index: {irregularity:.3f}", fontsize=14)
        plt.tight_layout()
        plt.show()

        return irregularity

    
    ### C - Color Variation
    def compute_color_variation(original_rgb_image, lesion_mask, n_clusters=5):
        # Resize mask to match image if needed
        if lesion_mask.shape != original_rgb_image.shape[:2]:
            lesion_mask = cv2.resize(lesion_mask.astype(np.uint8), original_rgb_image.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)

        # Extract only pixels inside the lesion
        lesion_pixels = original_rgb_image[lesion_mask > 0]

        if len(lesion_pixels) < n_clusters:
            print("Error. Segmented image too small")
            return

        # Run k-means clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels = kmeans.fit_predict(lesion_pixels)
        centers = kmeans.cluster_centers_

        # Compute intra-cluster variance (sum of squared distances)
        intra_variance = np.sum((lesion_pixels - centers[labels]) ** 2) / len(lesion_pixels)
        
        # Plot the centroids to visualise the variability of the colors 
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111, projection='3d')

        # Plot each centroid as a point
        for color in centers:
            ax.scatter(color[0], color[1], color[2],
                    color=color / 255.0,
                    s=200, edgecolor='k', label=f'RGB({int(color[0])}, {int(color[1])}, {int(color[2])})')

        ax.set_xlabel('Red')
        ax.set_ylabel('Green')
        ax.set_zlabel('Blue')
        ax.set_title('Color Cluster Centroids in RGB Space')
        ax.legend(loc='best')
        plt.tight_layout()
        plt.show()
        return n_clusters, intra_variance, centers.astype(np.uint8)

    ### D - Diameter
    def compute_diameter(mask):
        region = regionprops(mask.astype(int))[0]
        diameter = max(region.major_axis_length, region.minor_axis_length)
        return diameter

    asymmetry = plot_lesion_symmetry(lesion_mask)
    border_irregularity = plot_border_irregularity(lesion_mask, image)
    color_variation = compute_color_variation(image, lesion_mask)
    diameter = compute_diameter(lesion_mask)

    
visualize_abcd_features(img)